In [1]:
import argparse
import cv2
import io
import PIL.Image
from net import *

os.environ['CUDA_VISIBLE_DEVICES'] = GPU_ID

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

seed = 8964

# input image path
parser = argparse.ArgumentParser()

parser.add_argument('--phase', type=str, default='Test',
					help='Train/Test network.')

class MODEL(Network):
	"""docstring for MODEL"""
	def __init__(self):
		Network.__init__(self)
		self.log_dir = 'pretrained'
		self.eval_file = './dataset/cubicasa_clean_train.txt'		
		self.loss_type = 'balanced'

	def convert_one_hot_to_image(self, one_hot, dtype='float', act=None):
		if act == 'softmax':
			one_hot = tf.nn.softmax(one_hot, axis=-1)

		[n, h, w, c] = one_hot.shape.as_list()

		im = tf.reshape(tf.argmax(one_hot, axis=-1), [n, h, w, 1])
		if dtype == 'int':
			im = tf.cast(im, dtype=tf.uint8)
		else:
			im = tf.cast(im, dtype=tf.float32)
		return im

	def cross_two_tasks_weight(self, y1, y2):
		p1 = tf.reduce_sum(y1)
		p2 = tf.reduce_sum(y2)

		w1 = p2 / (p1 + p2)
		w2 = p1 / (p1 + p2)

		return w1, w2

	def balanced_entropy(self, x, y):
		# cliped_by_eps
		eps = 1e-6
		z = tf.nn.softmax(x)
		cliped_z = tf.clip_by_value(z, eps, 1-eps)
		log_z = tf.log(cliped_z)

		num_classes = y.shape.as_list()[-1]
		ind = tf.argmax(y, -1, output_type=tf.int32)
		# ind = tf.reshape(ind, shape=[1, 512, 512, 1]) # for debugging

		total = tf.reduce_sum(y) # total foreground pixels

		m_c = [] # index mask
		n_c = [] # each class foreground pixels
		for c in range(num_classes):
			m_c.append(tf.cast(tf.equal(ind, c), dtype=tf.int32))
			n_c.append(tf.cast(tf.reduce_sum(m_c[-1]), dtype=tf.float32))

		# compute count
		c = []
		for i in range(num_classes):
			c.append(total - n_c[i])
		tc = tf.add_n(c)

		# use for compute loss
		loss = 0.
		for i in range(num_classes): 
			w = c[i] / tc
			m_c_one_hot = tf.one_hot((i*m_c[i]), num_classes, axis=-1)
			y_c = m_c_one_hot*y

			loss += w*tf.reduce_mean(-tf.reduce_sum(y_c*log_z, axis=1)) 

		return (loss / num_classes) # mean 

	def train(self, loader_dict, num_batch, max_step=40000):
		images	 = loader_dict['images']
		labels_r_hot = loader_dict['label_rooms']
		labels_cw_hot  = loader_dict['label_boundaries']

		max_ep = max_step // num_batch
		print ('max_step = {}, max_ep = {}, num_batch = {}'.format(max_step, max_ep, num_batch))

		logits1, logits2 = self.forward(images, init_with_pretrain_vgg=False)

		if self.loss_type == 'balanced':
			# in-task loss balance
			loss1 = self.balanced_entropy(logits1, labels_r_hot) # multi classes balance
			loss2 = self.balanced_entropy(logits2, labels_cw_hot)
		else:
			loss1 = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits1, labels=labels_r_hot, name='bce1'))
			loss2 = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits2, labels=labels_cw_hot, name='bce2'))

		# compute cross loss balance weight
		w1, w2 = self.cross_two_tasks_weight(labels_r_hot, labels_cw_hot)
		loss = (w1*loss1 + w2*loss2)

		optim = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(loss, colocate_gradients_with_ops=True) # gradient ops assign to same device as forward ops

		# define session
		config = tf.ConfigProto(allow_soft_placement=True) 
		config.gpu_options.allow_growth=True # prevent the program occupies all GPU memory
		with tf.Session(config=config) as sess:
			# init all variables in graph
			sess.run(tf.group(tf.global_variables_initializer(),
							tf.local_variables_initializer()))

			# saver 
			saver = tf.train.Saver(max_to_keep=10) 

			# coordinator for queue runner
			coord = tf.train.Coordinator()

			# start queue 
			threads = tf.train.start_queue_runners(sess=sess, coord=coord)

			print ("Start Training!")
			total_times = 0			

			for ep in range(max_ep): # epoch loop
				for n in range(num_batch): # batch loop
					tic = time.time()
					# [loss_value, update_value, summaries] = sess.run([loss, optim, merged])	
					[loss_value, update_value] = sess.run([loss, optim])	
					duration = time.time()-tic

					total_times += duration

					step = int(ep*num_batch + n)
					# write log 
					print ('step {}: loss = {:.3}; {:.2} data/sec, excuted {} minutes'.format(step,
						loss_value, 1.0/duration, int(total_times/60)))
					# writer.add_summary(summaries, global_step=step)
				# save model parameters after 2 epoch training
				if ep % 2 == 0:
					saver.save(sess, self.log_dir+'/model', global_step=ep)
					self.evaluate(sess=sess, epoch=ep)
			saver.save(sess, self.log_dir+'/model', global_step=max_ep)
			self.evaluate(sess=sess, epoch=max_ep)

			# close session
			coord.request_stop()
			coord.join(threads)			
			sess.close()	

	def infer(self, save_dir='out', resize=True, merge=True):
		print ("generating test set of {}.... will save to [./{}]".format(self.eval_file, save_dir))
		room_dir = os.path.join(save_dir, 'room')
		close_wall_dir = os.path.join(save_dir, 'boundary')

		if not os.path.exists(save_dir):
			os.mkdir(save_dir)
		if not os.path.exists(room_dir):
			os.mkdir(room_dir)
		if not os.path.exists(close_wall_dir):
			os.mkdir(close_wall_dir)

		x = tf.placeholder(shape=[1, 512, 512, 3], dtype=tf.float32)

		logits1, logits2 = self.forward(x, init_with_pretrain_vgg=False)
		rooms = self.convert_one_hot_to_image(logits1, act='softmax', dtype='int')
		close_walls = self.convert_one_hot_to_image(logits2, act='softmax', dtype='int')

		config = tf.ConfigProto(allow_soft_placement=True)
		sess = tf.Session(config=config)
		sess.run(tf.group(tf.global_variables_initializer(),
						tf.local_variables_initializer()))

		saver = tf.train.Saver() # restore all parameters
		saver.restore(sess, save_path = tf.train.latest_checkpoint(self.log_dir))

		# infer one by one
		paths = open(self.eval_file, 'r').read().splitlines()
		paths = [p.split('\t')[0] for p in paths]	
		for p in paths:

			# load input
			im_buf = PIL.Image.open(p)  
			resize = 512,512

			# generating the thumbnail from given size
			im_buf.thumbnail(resize, PIL.Image.ANTIALIAS)

			offset_x = max((resize[0] - im_buf.size[0]) / 2, 0)
			offset_y = max((resize[1] - im_buf.size[1]) / 2, 0)

			offset_tuple = (int(offset_x), int(offset_y)) #pack x and y into a tuple

			# create the image object to be the final product
			im_resized = PIL.Image.new(mode='RGB',size=resize,color=(255, 255, 255))

			# paste the thumbnail into the full sized image
			im_resized.paste(im_buf, offset_tuple)

			im = np.array(im_resized) / 255. # array and normalize the image
            
			im_x = np.reshape(im, (1,512,512,3))
            
			[out1, out2] = sess.run([rooms, close_walls], feed_dict={x: im_x})
			if resize:
				out1_rgb = ind2rgb(np.squeeze(out1))
				out1_rgb = imresize(out1_rgb, (im.shape[0], im.shape[1])) # resize back 
				out2_rgb = ind2rgb(np.squeeze(out2), color_map=floorplan_boundary_map)
				out2_rgb = imresize(out2_rgb, (im.shape[0], im.shape[1])) # resize back 
			else:
				out1_rgb = ind2rgb(np.squeeze(out1))
				out2_rgb = ind2rgb(np.squeeze(out2), color_map=floorplan_boundary_map)

			if merge:
				out1 = np.squeeze(out1)
				out2 = np.squeeze(out2)
				out1[out2==2] = 10
				out1[out2==1] = 9
				# out3_rgb = ind2rgb(out1, color_map=floorplan_fuse_map_figure) # use for present
				out3_rgb = ind2rgb(out1, color_map=floorplan_fuse_map) # use for present

			name = p.split('/')[-1]	
			save_path1 = os.path.join(room_dir, name.split('.jpg')[0]+'_rooms.png')		
			save_path2 = os.path.join(close_wall_dir, name.split('.jpg')[0]+'_bd_rm.png')		
			save_path3 = os.path.join(save_dir, name.split('.jpg')[0]+'_rooms.png')		

			imsave(save_path1, out1_rgb)
			imsave(save_path2, out2_rgb)
			if merge:
				imsave(save_path3, out3_rgb)
			# imsave(save_path4, out4)
			
			print ('Saving prediction: {}'.format(name))

	def evaluate(self, sess, epoch, num_of_classes=11):
		x = tf.placeholder(shape=[1, 512, 512, 3], dtype=tf.float32)
		logits1, logits2 = self.forward(x, init_with_pretrain_vgg=False)
		predict_bd = self.convert_one_hot_to_image(logits2, act='softmax', dtype='int')
		predict_room = self.convert_one_hot_to_image(logits1, act='softmax', dtype='int')

		paths = open(self.eval_file, 'r').read().splitlines()
		image_paths = [p.split('\t')[0] for p in paths] # image 
		gt2_paths = [p.split('\t')[2] for p in paths] # 2 denote doors (and windows)
		gt3_paths = [p.split('\t')[3] for p in paths] # 3 denote rooms
		gt4_paths = [p.split('\t')[-1] for p in paths] # last one denote close wall

		n = len(paths)

		hist = np.zeros((num_of_classes, num_of_classes))
		for i in range(n):
			im = imread(image_paths[i], mode='RGB')
			# for fuse label
			dd = imread(gt2_paths[i], mode='L')
			rr = imread(gt3_paths[i], mode='RGB')
			cw = imread(gt4_paths[i], mode='L')

			im = imresize(im, (512, 512, 3)) / 255. # normalize input image
			im = np.reshape(im, (1,512,512,3))
			# merge label
			rr = imresize(rr, (512, 512, 3))
			rr_ind = rgb2ind(rr)
			cw = imresize(cw, (512, 512)) / 255
			dd = imresize(dd, (512, 512)) / 255
			cw = (cw>0.5).astype(np.uint8)
			dd = (dd>0.5).astype(np.uint8)
			rr_ind[cw==1] = 10
			rr_ind[dd==1] = 9

			# merge prediciton
			rm_ind, bd_ind = sess.run([predict_room, predict_bd], feed_dict={x: im})
			rm_ind = np.squeeze(rm_ind)
			bd_ind = np.squeeze(bd_ind)
			rm_ind[bd_ind==2] = 10
			rm_ind[bd_ind==1] = 9

			hist += fast_hist(rm_ind.flatten(), rr_ind.flatten(), num_of_classes)

		overall_acc = np.diag(hist).sum() / hist.sum()
		mean_acc = np.diag(hist) / (hist.sum(1) + 1e-6)
		# iu = np.diag(hist) / (hist.sum(1) + 1e-6 + hist.sum(0) - np.diag(hist))			
		mean_acc9 = (np.nansum(mean_acc[:7])+mean_acc[-2]+mean_acc[-1]) / 9.

		file = open('EVAL_'+self.log_dir, 'a')
		print(file, 'Model at epoch {}: overall accuracy = {:.4}, mean_acc = {:.4}'.format(epoch, overall_acc, mean_acc9))
		for i in range(mean_acc.shape[0]):
			if i not in [7 ,8]: # ingore class 7 & 8 
				print(file, '\t\tepoch {}: {}th label: accuracy = {:.4}'.format(epoch, i, mean_acc[i]))
		file.close()

def main(args):
	tf.set_random_seed(seed)
	np.random.seed(seed)
	random.seed(seed)

	model = MODEL()

	if args.phase.lower() == 'train':
		loader_dict, num_batch = data_loader_bd_rm_from_tfrecord(batch_size=1)

		# START TRAINING
		tic = time.time()
		model.train(loader_dict, num_batch)
		toc = time.time()
		print('total training + evaluation time = {} minutes'.format((toc-tic)/60))
	elif args.phase.lower() == 'test':	
		model.infer()
	else:
		pass

if __name__ == '__main__':
	FLAGS, unparsed = parser.parse_known_args()
	main(FLAGS)


E:\Python\Anaconda\envs\TFGPU\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
E:\Python\Anaconda\envs\TFGPU\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
E:\Python\Anaconda\envs\TFGPU\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
E:\Python\Anaconda\envs\TFGPU\lib\site-packages\tensorflow\python\framework\dtypes.py:526: 

Initial nn network object...
generating test set of ./dataset/cubicasa_clean_train.txt.... will save to [./out]
False
INFO:tensorflow:Restoring parameters from pretrained\pretrained_r3d


E:\Python\Anaconda\envs\TFGPU\lib\site-packages\ipykernel_launcher.py:211: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.
E:\Python\Anaconda\envs\TFGPU\lib\site-packages\ipykernel_launcher.py:213: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.
E:\Python\Anaconda\envs\TFGPU\lib\site-packages\ipykernel_launcher.py:231: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
E:\Python\Anaconda\envs\TFGPU\lib\site-packages\ipykernel_launcher.py:232: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
E:\Python\Anaconda\envs\TFGPU\lib\site

Saving prediction: 1.jpg
Saving prediction: 2.jpg
Saving prediction: 3.jpg
Saving prediction: 4.jpg
Saving prediction: 5.jpg
Saving prediction: 6.jpg
Saving prediction: 7.jpg
Saving prediction: 8.jpg
Saving prediction: 9.jpg
Saving prediction: 10.jpg
Saving prediction: 11.jpg
Saving prediction: 12.jpg
Saving prediction: 13.jpg
Saving prediction: 14.jpg
Saving prediction: 15.jpg
Saving prediction: 16.jpg
Saving prediction: 17.jpg
Saving prediction: 18.jpg
Saving prediction: 19.jpg
Saving prediction: 20.jpg
Saving prediction: 21.jpg
Saving prediction: 22.jpg
Saving prediction: 23.jpg
Saving prediction: 24.jpg
Saving prediction: 25.jpg
Saving prediction: 26.jpg
Saving prediction: 27.jpg
Saving prediction: 28.jpg
Saving prediction: 29.jpg
Saving prediction: 30.jpg
Saving prediction: 31.jpg
Saving prediction: 32.jpg
Saving prediction: 33.jpg
Saving prediction: 34.jpg
Saving prediction: 35.jpg
Saving prediction: 36.jpg
Saving prediction: 37.jpg
Saving prediction: 38.jpg
Saving prediction: 39

Saving prediction: 309.jpg
Saving prediction: 310.jpg
Saving prediction: 311.jpg
Saving prediction: 312.jpg
Saving prediction: 313.jpg
Saving prediction: 314.jpg
Saving prediction: 315.jpg
Saving prediction: 316.jpg
Saving prediction: 317.jpg
Saving prediction: 318.jpg
Saving prediction: 319.jpg
Saving prediction: 320.jpg
Saving prediction: 321.jpg
Saving prediction: 322.jpg
Saving prediction: 323.jpg
Saving prediction: 324.jpg
Saving prediction: 325.jpg
Saving prediction: 326.jpg
Saving prediction: 327.jpg
Saving prediction: 328.jpg
Saving prediction: 329.jpg
Saving prediction: 330.jpg
Saving prediction: 331.jpg
Saving prediction: 332.jpg
Saving prediction: 333.jpg
Saving prediction: 334.jpg
Saving prediction: 335.jpg
Saving prediction: 336.jpg
Saving prediction: 337.jpg
Saving prediction: 338.jpg
Saving prediction: 339.jpg
Saving prediction: 340.jpg
Saving prediction: 341.jpg
Saving prediction: 342.jpg
Saving prediction: 343.jpg
Saving prediction: 344.jpg
Saving prediction: 345.jpg
S

...

Saving prediction: 3850.jpg
Saving prediction: 3851.jpg
Saving prediction: 3852.jpg
Saving prediction: 3853.jpg
Saving prediction: 3854.jpg
Saving prediction: 3855.jpg
Saving prediction: 3856.jpg
Saving prediction: 3857.jpg
Saving prediction: 3858.jpg
Saving prediction: 3859.jpg
Saving prediction: 3860.jpg
Saving prediction: 3861.jpg
Saving prediction: 3862.jpg
Saving prediction: 3863.jpg
Saving prediction: 3864.jpg
Saving prediction: 3865.jpg
Saving prediction: 3866.jpg
Saving prediction: 3867.jpg
Saving prediction: 3868.jpg
Saving prediction: 3869.jpg
Saving prediction: 3870.jpg
Saving prediction: 3871.jpg
Saving prediction: 3872.jpg
Saving prediction: 3873.jpg
Saving prediction: 3874.jpg
Saving prediction: 3875.jpg
Saving prediction: 3876.jpg
Saving prediction: 3877.jpg
Saving prediction: 3878.jpg
Saving prediction: 3879.jpg
Saving prediction: 3880.jpg
Saving prediction: 3881.jpg
Saving prediction: 3882.jpg
Saving prediction: 3883.jpg
Saving prediction: 3884.jpg
Saving prediction: 3

Saving prediction: 4143.jpg
Saving prediction: 4144.jpg
Saving prediction: 4145.jpg
Saving prediction: 4146.jpg
Saving prediction: 4147.jpg
Saving prediction: 4148.jpg
Saving prediction: 4149.jpg
Saving prediction: 4150.jpg
Saving prediction: 4151.jpg
Saving prediction: 4152.jpg
Saving prediction: 4153.jpg
Saving prediction: 4154.jpg
Saving prediction: 4155.jpg
Saving prediction: 4156.jpg
Saving prediction: 4157.jpg
Saving prediction: 4158.jpg
Saving prediction: 4159.jpg
Saving prediction: 4160.jpg
Saving prediction: 4161.jpg
Saving prediction: 4162.jpg
Saving prediction: 4163.jpg
Saving prediction: 4164.jpg
Saving prediction: 4165.jpg
Saving prediction: 4166.jpg
Saving prediction: 4167.jpg
Saving prediction: 4168.jpg
Saving prediction: 4169.jpg
Saving prediction: 4170.jpg
Saving prediction: 4171.jpg
Saving prediction: 4172.jpg
Saving prediction: 4173.jpg
Saving prediction: 4174.jpg
Saving prediction: 4175.jpg
Saving prediction: 4176.jpg
Saving prediction: 4177.jpg
Saving prediction: 4

Saving prediction: 4436.jpg
Saving prediction: 4437.jpg
Saving prediction: 4438.jpg
Saving prediction: 4439.jpg
Saving prediction: 4440.jpg
Saving prediction: 4441.jpg
Saving prediction: 4442.jpg
Saving prediction: 4443.jpg
Saving prediction: 4444.jpg
Saving prediction: 4445.jpg
Saving prediction: 4446.jpg
Saving prediction: 4447.jpg
Saving prediction: 4448.jpg
Saving prediction: 4449.jpg
Saving prediction: 4450.jpg
Saving prediction: 4451.jpg
Saving prediction: 4452.jpg
Saving prediction: 4453.jpg
Saving prediction: 4454.jpg
Saving prediction: 4455.jpg
Saving prediction: 4456.jpg
Saving prediction: 4457.jpg
Saving prediction: 4458.jpg
Saving prediction: 4459.jpg
Saving prediction: 4460.jpg
Saving prediction: 4461.jpg
Saving prediction: 4462.jpg
Saving prediction: 4463.jpg
Saving prediction: 4464.jpg
Saving prediction: 4465.jpg
Saving prediction: 4466.jpg
Saving prediction: 4467.jpg
Saving prediction: 4468.jpg
Saving prediction: 4469.jpg
Saving prediction: 4470.jpg
Saving prediction: 4

Saving prediction: 4729.jpg
Saving prediction: 4730.jpg
Saving prediction: 4731.jpg
Saving prediction: 4732.jpg
Saving prediction: 4733.jpg
Saving prediction: 4734.jpg
Saving prediction: 4735.jpg
Saving prediction: 4736.jpg
Saving prediction: 4737.jpg
Saving prediction: 4738.jpg
Saving prediction: 4739.jpg
Saving prediction: 4740.jpg
Saving prediction: 4741.jpg
Saving prediction: 4742.jpg
Saving prediction: 4743.jpg
Saving prediction: 4744.jpg
Saving prediction: 4745.jpg
Saving prediction: 4746.jpg
Saving prediction: 4747.jpg
Saving prediction: 4748.jpg
Saving prediction: 4749.jpg
Saving prediction: 4750.jpg
Saving prediction: 4751.jpg
Saving prediction: 4752.jpg
Saving prediction: 4753.jpg
Saving prediction: 4754.jpg
Saving prediction: 4755.jpg
Saving prediction: 4756.jpg
Saving prediction: 4757.jpg
Saving prediction: 4758.jpg
Saving prediction: 4759.jpg
Saving prediction: 4760.jpg
Saving prediction: 4761.jpg
Saving prediction: 4762.jpg
Saving prediction: 4763.jpg
Saving prediction: 4

Saving prediction: 5022.jpg
Saving prediction: 5023.jpg
Saving prediction: 5024.jpg
Saving prediction: 5025.jpg
Saving prediction: 5026.jpg
Saving prediction: 5027.jpg
Saving prediction: 5028.jpg
Saving prediction: 5029.jpg
Saving prediction: 5030.jpg
Saving prediction: 5031.jpg
Saving prediction: 5032.jpg
Saving prediction: 5033.jpg
Saving prediction: 5034.jpg
Saving prediction: 5035.jpg
Saving prediction: 5036.jpg
Saving prediction: 5037.jpg
Saving prediction: 5038.jpg
Saving prediction: 5039.jpg
Saving prediction: 5040.jpg
Saving prediction: 5041.jpg
Saving prediction: 5042.jpg
Saving prediction: 5043.jpg
Saving prediction: 5044.jpg
Saving prediction: 5045.jpg
Saving prediction: 5046.jpg
Saving prediction: 5047.jpg
Saving prediction: 5048.jpg
Saving prediction: 5049.jpg
Saving prediction: 5050.jpg
Saving prediction: 5051.jpg
Saving prediction: 5052.jpg
Saving prediction: 5053.jpg
Saving prediction: 5054.jpg
Saving prediction: 5055.jpg
Saving prediction: 5056.jpg
Saving prediction: 5